In [ ]:
import dgl
from dgl import DGLGraph
from dgl.data import citegrh
from dgl.nn.pytorch import conv

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class GraphSAGE(nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.g = g

        # input layer
        self.layers.append(conv.SAGEConv(in_feats, n_hidden, aggregator_type, feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(conv.SAGEConv(n_hidden, n_hidden, aggregator_type, feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(conv.SAGEConv(n_hidden, n_classes, aggregator_type, feat_drop=dropout, activation=None)) # activation None

    def forward(self, features):
        h = features
        for layer in self.layers:
            h = layer(self.g, h)
        return h


## Load data

In [ ]:
# load and preprocess dataset
data = citegrh.load_pubmed()
features = torch.FloatTensor(data.features)
labels = torch.LongTensor(data.labels)
train_mask = torch.ByteTensor(data.train_mask)
val_mask = torch.ByteTensor(data.val_mask)
test_mask = torch.ByteTensor(data.test_mask)
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = data.graph.number_of_edges()
print("""----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (n_edges, n_classes,
           train_mask.sum().item(),
           val_mask.sum().item(),
           test_mask.sum().item()))

In [ ]:
g = data.graph
print(g.number_of_nodes(), g.number_of_edges())
g.remove_edges_from(g.selfloop_edges())
print(g.number_of_nodes(), g.number_of_edges())
g = DGLGraph(g)
g.readonly()

## Hyperparameters

In [ ]:
n_hidden = 16
n_layers = 2
dropout = 0.5
aggregator_type = 'gcn'
weight_decay = 5e-4
n_epochs = 2000

## Node classification task

In [ ]:
# create GraphSAGE model
gconv_model = GraphSAGE(g,
                        in_feats,
                        n_hidden,
                        n_classes,
                        n_layers,
                        F.relu,
                        dropout,
                        aggregator_type)

class NodeClassification(nn.Module):
    def __init__(self, gconv_model):
        super(NodeClassification, self).__init__()
        self.gconv_model = gconv_model
        self.loss_fcn = torch.nn.CrossEntropyLoss()

    def forward(self, features, train_mask):
        logits = self.gconv_model(features)
        return self.loss_fcn(logits[train_mask], labels[train_mask])

# Node classification task
model = NodeClassification(gconv_model)
    
def evaluate(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = gconv_model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

## Link prediction

We first split the graph into the training set and the testing set.

In [ ]:
eids = np.random.permutation(g.number_of_edges())
train_eids = eids[:int(len(eids) * 0.8)]
test_eids = eids[int(len(eids) * 0.8):]
train_g = g.edge_subgraph(train_eids, preserve_nodes=True)
test_g = g.edge_subgraph(test_eids, preserve_nodes=True)

Construct negative edges for training

In [ ]:
def neg_sample(g, neg_sample_size, edges=None):
    sampler = dgl.contrib.sampling.EdgeSampler(g, batch_size=g.number_of_edges(),
                                               seed_edges=edges,
                                               neg_sample_size=neg_sample_size,
                                               negative_mode='tail',
                                               return_false_neg=True)
    sampler = iter(sampler)
    return next(sampler)

In [ ]:
#hyperparameters
neg_sample_size = 100
lr = 1e-1
n_layers = 1

# create GraphSAGE model
gconv_model = GraphSAGE(train_g,
                        in_feats,
                        n_hidden,
                        16,
                        n_layers,
                        F.relu,
                        dropout,
                        aggregator_type)

class LinkPrediction(nn.Module):
    def __init__(self, gconv_model):
        super(LinkPrediction, self).__init__()
        self.gconv_model = gconv_model

    def forward(self, features, train_mask):
        emb = self.gconv_model(features)
        pos_g, neg_g = neg_sample(g, neg_sample_size)
        pos_src, pos_dst = pos_g.all_edges()
        pos_heads = emb[pos_src]
        pos_tails = emb[pos_dst]
        neg_src, neg_dst = neg_g.all_edges()
        neg_heads = emb[neg_src].reshape(-1, neg_sample_size, emb.shape[1])
        neg_tails = emb[neg_dst].reshape(-1, neg_sample_size, emb.shape[1])
        assert neg_heads.shape[0] == neg_tails.shape[0]
        pos_score = F.logsigmoid(torch.sum(pos_heads * pos_tails, dim=1))
        neg_score = F.logsigmoid(-torch.sum(neg_heads * neg_tails, dim=2))
        return torch.mean(-pos_score - torch.sum(neg_score, dim=1))
    

# Link prediction task
model = LinkPrediction(gconv_model)

def evaluate(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        emb = gconv_model(features)
        
        pos_g, neg_g = neg_sample(g, neg_sample_size, test_eids)
        pos_src, pos_dst = pos_g.all_edges()
        pos_heads = emb[pos_src]
        pos_tails = emb[pos_dst]
        neg_src, neg_dst = neg_g.all_edges()
        neg_heads = emb[neg_src].reshape(-1, neg_sample_size, emb.shape[1])
        neg_tails = emb[neg_dst].reshape(-1, neg_sample_size, emb.shape[1])
        filter_bias = neg_g.edata['false_neg'].reshape(-1, neg_sample_size)

        pos_score = F.logsigmoid(torch.sum(pos_heads * pos_tails, dim=1))
        neg_score = F.logsigmoid(torch.sum(neg_heads * neg_tails, dim=2))
        neg_score -= filter_bias.float()
        pos_score = pos_score.unsqueeze(1)
        rankings = torch.sum(neg_score > pos_score, dim=1) + 1
        print('MR:', np.mean(rankings.numpy()))
        print('MRR:', np.mean(1.0/rankings.numpy()))
        return np.mean(1.0/rankings.numpy())

The training loop

In [ ]:
# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
for epoch in range(n_epochs):
    model.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    loss = model(features, train_mask)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    acc = evaluate(model, features, labels, val_mask)
    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur), loss.item(),
                                            acc, n_edges / np.mean(dur) / 1000))

print()
acc = evaluate(model, features, labels, test_mask)
print("Test Accuracy {:.4f}".format(acc))
